In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.5.0'

### Basic Graph

In [40]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

### Basic Session

In [41]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
#     x.initializer.run()
#     tf.get_default_session().run(x.initializer)
#     sess.run(x.initializer)
    
#     y.initializer.run()
#     tf.get_default_session().run(y.initializer)
#     sess.run(y.initializer)

    init.run()
    
    result = sess.run(f)
    print(result)

42


### Intertactive Session

In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


### Managing Graphs

In [10]:
# Variables are automatically added to default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
# Define graph by yourselves
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

# graph is no longer default graph because it is out of the block
x2.graph is tf.get_default_graph()

False

In [42]:
# Reset graph
tf.reset_default_graph()

### Lifecycle of a Node Value

In [45]:
w = tf.constant(3)

x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval()) # 10
    # For this evaluation, it calculates x & y again.
    print(z.eval()) # 15
    
    # To avoid redundant calculation:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15
10
15


## Linear Regression with TensorFlow

In [4]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [74]:
def normal_equation_model():    
    X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
    XT = tf.transpose(X)
    theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
    return theta

In [76]:
theta = normal_equation_model()

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


### Gradient Descent

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [6]:
n_epochs = 1000
learning_rate = 0.01

In [77]:
def gradient_model():
    tf.reset_default_graph()

    X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

    theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")

    y_pred = tf.matmul(X, theta, name="predictions")
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

    # 1. Manually calculated gradients
    # gradients = 2/m * tf.matmul(tf.transpose(X), error)
    # training_op = tf.assign(theta, theta - learning_rate * gradients)

    # 2. Automatically calculated gradients by TensorFlow
    # gradients = tf.gradients(mse, [theta])[0]
    # training_op = tf.assign(theta, theta - learning_rate * gradients)

    # 3. Using optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)
    
    return theta, mse, training_op

In [78]:
theta, mse, training_op = gradient_model()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE =  7.23475
Epoch 100 MSE =  4.86437
Epoch 200 MSE =  4.83649
Epoch 300 MSE =  4.82686
Epoch 400 MSE =  4.82034
Epoch 500 MSE =  4.81565
Epoch 600 MSE =  4.81226
Epoch 700 MSE =  4.8098
Epoch 800 MSE =  4.80802
Epoch 900 MSE =  4.80673
[[ 0.62484741]
 [ 0.81808275]
 [ 0.1355291 ]
 [-0.20830411]
 [ 0.24279805]
 [ 0.00174982]
 [-0.04049946]
 [-0.78841257]
 [-0.75601125]]


### Using Mini-Batch & Saving Models

In [2]:
def mini_batch_gradient_model():
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

    theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")

    y_pred = tf.matmul(X, theta, name="predictions")
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)
    
    return X, y, theta, mse, training_op

In [11]:
def fetch_batch(batch_index, batch_size):
    startIdx = batch_index * batch_size
    endIdx = startIdx + batch_size
    return (scaled_housing_data_plus_bias[startIdx:endIdx],
            housing.target.reshape(-1, 1)[startIdx:endIdx])

In [88]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X, y, theta, mse, training_op = mini_batch_gradient_model()

init = tf.global_variables_initializer()

# Saving models
saver = tf.train.Saver()
# If you need to save only theta variables:
# saver = tf.train.Saver({"weights": theta})

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):          
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", sess.run(mse, feed_dict={X: X_batch, y: y_batch}))
            save_path = saver.save(sess, "tmp/my_model.ckpt")
            
    best_theta = theta.eval()
    save_path = saver.save(sess, "tmp/my_model_final.ckpt")
    print(best_theta)

Epoch 0 MSE =  0.222841
Epoch 100 MSE =  0.453871
Epoch 200 MSE =  0.453871
Epoch 300 MSE =  0.453871
Epoch 400 MSE =  0.453871
Epoch 500 MSE =  0.453871
Epoch 600 MSE =  0.453871
Epoch 700 MSE =  0.453871
Epoch 800 MSE =  0.453871
Epoch 900 MSE =  0.453871
[[ -5.29047012e-01]
 [  9.95532930e-01]
 [  9.72431973e-02]
 [ -3.68002534e-01]
 [  3.56470287e-01]
 [ -3.56888422e-03]
 [  9.67856147e-04]
 [ -2.38165140e-01]
 [ -1.48855364e+00]]


### Restoring Models

In [3]:
# _, _, theta, _, _ = mini_batch_gradient_model()
# saver = tf.train.Saver()

# Or you can just load the model from *.meta file
# and add them to default graph
saver = tf.train.import_meta_graph("tmp/my_model_final.ckpt.meta")


with tf.Session() as sess:
    # Don't need to init variables or train models
    # because you will resotre model from the last checkpoint
    saver.restore(sess, "tmp/my_model_final.ckpt")
    theta = tf.get_default_graph().get_tensor_by_name("theta:0")
    best_theta = sess.run(theta)
    print(best_theta)
    

INFO:tensorflow:Restoring parameters from tmp/my_model_final.ckpt
[[ -5.29047012e-01]
 [  9.95532930e-01]
 [  9.72431973e-02]
 [ -3.68002534e-01]
 [  3.56470287e-01]
 [ -3.56888422e-03]
 [  9.67856147e-04]
 [ -2.38165140e-01]
 [ -1.48855364e+00]]


### Using Name Scopes & TensorBoard

In [16]:
from datetime import datetime

def nowstr():
    return datetime.utcnow().strftime("%Y%m%d%H%M%S")

In [17]:
def mini_batch_gradient_model_with_name_scopes_and_stats():
    root_logdir = "tf_logs"
    logdir = "{}/run-{}/".format(root_logdir, nowstr())
    
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

    theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")

    y_pred = tf.matmul(X, theta, name="predictions")
    
    with tf.name_scope("loss") as scope:
        error = y_pred - y
        mse = tf.reduce_mean(tf.square(error), name="mse")

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)
    
    mse_summary = tf.summary.scalar("MSE", mse)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    return X, y, theta, mse_summary, training_op, file_writer

In [15]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X, y, theta, mse_summary, training_op, file_writer = mini_batch_gradient_model_with_name_scopes_and_stats()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()
    print(best_theta)

file_writer.close()

[[  7.98887968e-01]
 [  9.95532930e-01]
 [  9.72431973e-02]
 [ -3.68002534e-01]
 [  3.56470287e-01]
 [ -3.56888422e-03]
 [  9.67856147e-04]
 [ -2.38165140e-01]
 [ -1.48855364e+00]]


## Modularity

### Name Scope

In [18]:
def relu_name_scope(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")

        return tf.maximum(z, 0., name="relu")

In [20]:
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu_name_scope(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tf_logs/relus-{}".format(nowstr()), tf.get_default_graph())
file_writer.close()

### Variable Scope

In [21]:
def relu_variable_scope(X):
    with tf.variable_scope("relu", reuse=True):
        # "reuse=True" above or scope.reuse_variables()
        # Reuse mode cannot be reverted inside the block.
        
        # If threshold has not been defined earlier in "relu" variable scope, it will raise exception.
        # Note that you can use this functionality when you only use tf.get_variable()
        threshold = tf.get_variable("threshold")
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")

        return tf.maximum(z, threshold, name="relu")

In [24]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                               initializer=tf.constant_initializer(0.0))
relus = [relu_variable_scope(X) for relu_index in range(5)]
output = tf.add_n(relus, name="outout")

file_writer = tf.summary.FileWriter("tf_logs/relus-{}".format(nowstr()), tf.get_default_graph())
file_writer.close()